In [1]:
import pymorphy2
import re
from collections import Counter
import pandas
morph = pymorphy2.MorphAnalyzer()
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect('words.db')
cur = conn.cursor()

In [3]:
cur.execute("""
CREATE TABLE IF NOT EXISTS pos_tag_lemma 
(lemma text, 
word text,
pos text, 
tag text)
""")
conn.commit()

In [4]:
def find_lemma(word):
    p = morph.parse(word)
    ls = []
    for s in p:
        l = [s.normal_form, word, s.tag.POS, ' '.join(re.findall('\w+', str(s.tag))[1:])]
        ls.append(l)

    return ls

In [5]:
find_lemma('август')

[['август', 'август', 'NOUN', 'inan masc sing nomn'],
 ['август', 'август', 'NOUN', 'anim masc Name sing nomn'],
 ['август', 'август', 'NOUN', 'inan masc sing accs'],
 ['августа', 'август', 'NOUN', 'anim femn Name plur gent'],
 ['августа', 'август', 'NOUN', 'anim femn Name plur accs']]

In [6]:
cur.execute('''
            select word
            from main
            order by freq DESC
            limit 20000''')
w = cur.fetchall()
for row in w:
    word = row[0]
    ls = find_lemma(word)
    for l in ls:
        cur.execute('''
                    insert into pos_tag_lemma (lemma, word, pos, tag)
                    values (?, ?, ?, ?)''', (l[0], l[1], l[2], l[3]))
        conn.commit()